# A. Librerias y rutas

In [1]:
import os
import time
import json
import platform
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 1. Definir rutas y sistema operativo
os_platform = platform.system().lower()

# Definir la carpeta de descargas para los archivos JSON
LOG_FILE_PATH = os.path.join("./scraping_log.txt")

# 2. Configurar rutas del proyecto según el sistema operativo
if os_platform == "darwin":
    CHROMEDRIVER_BASE_PATH = "/Volumes/Projects/GitHubProjects/UNI-SCRAPING/chromedriver"
elif os_platform == "linux":
    CHROMEDRIVER_BASE_PATH = "/home/brew_test_gcp_01/Desktop/WallyScraper/chromedriver"
elif os_platform == "windows":
    CHROMEDRIVER_BASE_PATH = "D:\\GithubProjects\\Tesis\\Transparencia\\chromedriver"

if os_platform == "darwin":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "darwin", "chromedriver")
elif os_platform == "linux":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "linux", "chromedriver")
elif os_platform == "windows":
    CHROMEDRIVER_PATH = os.path.join(CHROMEDRIVER_BASE_PATH, "windows", "chromedriver.exe")


# 3. Imprimir variables
print("ESTAMOS EN ---> ", os_platform)
print("CHROME_DRIVER_PATH: ", CHROMEDRIVER_PATH)
print("")

ESTAMOS EN --->  darwin
CHROME_DRIVER_PATH:  /Volumes/Projects/GitHubProjects/UNI-SCRAPING/chromedriver/darwin/chromedriver



# B. Funciones

In [3]:
def get_chrome_driver(chromedriver_path, print_view=False, headless=False):
    # Configurar las opciones de Selenium
    options = webdriver.ChromeOptions()
    options.add_argument("--no-sandbox")
    options.add_argument("--start-maximized")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--ignore-certificate-errors")

    # Argumentos opcionales
    if print_view:
        options.add_argument("--disable-print-preview")
    if headless:
        options.add_argument("--headless=new")

    # Configurar el servicio del driver
    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    
    return driver

# C. Prueba

In [16]:
driver = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=False, headless=False)

In [17]:
url = 'https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=112163&estado=REAL&tipo_esta=M&cate=CO&cod_old='
driver.get(url)

In [18]:
# Click en el botón "Tabla" usando By.ID
driver.find_element(By.ID, "tabl").click()

In [19]:
# Extraer el html de la página
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Buscar el elemento del captcha por su tag y atributos característicos
# Típicamente el captcha está en un span con estilo de color rojo y tamaño grande
captcha_element = soup.find('span', {'style': 'color: red; font-size:xxx-large'})
if captcha_element:
    print("Captcha encontrado:", captcha_element.text)
else:
    print("Captcha no encontrado")


Captcha encontrado: Q5HPk5rP


In [20]:
# Encontrar el campo de entrada del captcha por su ID
captcha_input = driver.find_element(By.ID, "captchaInput")

# Ingresar el texto del captcha
captcha_input.send_keys(captcha_element.text)

In [21]:
# Click en el botón "Verificar" usando By.ID
driver.find_element(By.ID, "entrar").click()

In [28]:
driver.switch_to.frame("contenedor")  # Asegúrate de usar el ID o nombre correcto

In [29]:
driver.find_element(By.ID, "export101").click()

In [30]:
driver.quit()

# D. Automatizar

In [5]:
# leer ./results/urls.txt
with open('./results/urls.txt', 'r') as f:
    urls = f.readlines()

urls = [url.strip() for url in urls[3:7]]
urls

['https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=221110&estado=REAL&tipo_esta=M&cate=CO&cod_old=',
 'https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=100113&estado=REAL&tipo_esta=M&cate=CO&cod_old=',
 'https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=106077&estado=REAL&tipo_esta=M&cate=CO&cod_old=',
 'https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=220405&estado=REAL&tipo_esta=M&cate=CO&cod_old=']

In [36]:
# Procesar cada URL
for url in urls:
    try:
        # Iniciar el driver
        driver = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=False, headless=False)
        print(f"\nProcesando URL: {url}")
        
        # Cargar la página
        driver.get(url)
        time.sleep(5)
        
        # Click en el botón "Tabla"
        driver.find_element(By.ID, "tabl").click()
        time.sleep(3)
        
        # Extraer y procesar el captcha
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        captcha_element = soup.find('span', {'style': 'color: red; font-size:xxx-large'})
        
        if captcha_element:
            # Ingresar el captcha
            captcha_input = driver.find_element(By.ID, "captchaInput")
            captcha_input.send_keys(captcha_element.text)
            time.sleep(3)
            
            # Verificar captcha
            driver.find_element(By.ID, "entrar").click()
            time.sleep(3)
            
            # Cambiar al frame y exportar
            driver.switch_to.frame("contenedor")
            time.sleep(3)
            driver.find_element(By.ID, "export101").click()
            time.sleep(3)
        
        # Cerrar el driver
        driver.quit()
        
        # Esperar antes de la siguiente URL
        time.sleep(2)
        
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        if 'driver' in locals():
            driver.quit()
        time.sleep(2)
        continue


Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=221110&estado=REAL&tipo_esta=M&cate=CO&cod_old=

Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=100113&estado=REAL&tipo_esta=M&cate=CO&cod_old=

Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=106077&estado=REAL&tipo_esta=M&cate=CO&cod_old=

Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=220405&estado=REAL&tipo_esta=M&cate=CO&cod_old=


In [6]:
# Procesar cada URL
for url in urls:
    try:
        # Iniciar el driver
        driver = get_chrome_driver(chromedriver_path=CHROMEDRIVER_PATH, print_view=False, headless=False)
        print(f"\nProcesando URL: {url}")
        
        # Configurar wait
        wait = WebDriverWait(driver, 10)  # 10 segundos de timeout
        
        # Cargar la página
        driver.get(url)
        
        # Click en el botón "Tabla" - esperar que sea clickeable
        tabla_button = wait.until(EC.element_to_be_clickable((By.ID, "tabl")))
        tabla_button.click()
        
        # Esperar que aparezca el captcha
        captcha_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span[style*="color: red"][style*="font-size:xxx-large"]')))
        captcha_text = captcha_element.text
        
        if captcha_text:
            # Ingresar el captcha - esperar que el input esté presente
            captcha_input = wait.until(EC.presence_of_element_located((By.ID, "captchaInput")))
            captcha_input.send_keys(captcha_text)
            
            # Verificar captcha - esperar que el botón sea clickeable
            verify_button = wait.until(EC.element_to_be_clickable((By.ID, "entrar")))
            verify_button.click()
            
            # Cambiar al frame - esperar que esté disponible
            wait.until(EC.frame_to_be_available_and_switch_to_it("contenedor"))
            
            # Esperar y hacer click en exportar
            export_button = wait.until(EC.element_to_be_clickable((By.ID, "export101")))
            export_button.click()
            
            # Pequeña pausa para asegurar que la descarga inicie
            time.sleep(1)
        
        # Cerrar el driver
        driver.quit()
        
        # Breve pausa entre URLs para evitar sobrecarga
        time.sleep(1)
        
    except Exception as e:
        print(f"Error procesando {url}: {str(e)}")
        if 'driver' in locals():
            driver.quit()
        time.sleep(1)
        continue


Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=221110&estado=REAL&tipo_esta=M&cate=CO&cod_old=

Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=100113&estado=REAL&tipo_esta=M&cate=CO&cod_old=

Procesando URL: https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=106077&estado=REAL&tipo_esta=M&cate=CO&cod_old=
Error procesando https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/map_red_graf.php?cod=106077&estado=REAL&tipo_esta=M&cate=CO&cod_old=: Alert Text: Captcha distinto.
Message: unexpected alert open: {Alert text : Captcha distinto.}
  (Session info: chrome=133.0.6943.54)
Stacktrace:
0   chromedriver                        0x00000001049bf94c cxxbridge1$str$ptr + 2728228
1   chromedriver                        0x00000001049b7fa8 cxxbridge1$str$ptr + 2697088
2   chromedriver                        0x000000010451dff0 cxxbridge1$string$len + 93348
3   chromedriver                        